In [20]:
import os
import pandas as pd
import numpy as np


print("Versions:")
print(f"{' '*2}- Pandas =>: {pd.__version__}")
print(f"{' '*2}- Numpy  =>: {np.__version__}")

Versions:
  - Pandas =>: 2.2.3
  - Numpy  =>: 1.26.4
